# Top Sales Image Retrieval

### Purpose: Merge master URL dataset from theGamesDB.net APIs to VGChartz dataset of sales and reviews and analyze top sold games.  Then use webscraping to retrieve and resize top 2600 games with URL result list. 

In [2]:
import pandas as pd
import os
import urllib
from urllib.request import Request, urlopen
import numpy as np
import PIL
from PIL import Image

In [3]:
# Read master piclist csv including all platform API results combined in other notebook
allsys_df = pd.read_csv('piclists/master_piclist.csv')

In [4]:
# Show top dataframe rows
allsys_df.head()

,gdb_id,Title,Release_Date,Thumb,Platform,URL
0,3373,Dark Arena,1/18/02,boxart/original/front/3373-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...
1,3374,Dave Mirra Freestyle BMX 2,11/25/01,boxart/original/front/3374-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...
2,3375,Davis Cup Tennis,10/14/02,boxart/original/front/3375-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...
3,198,Classic NES Series: Zelda II: The Adventure of...,10/25/04,boxart/original/front/198-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...
4,45987,Dragon Drive: World D Break,7/18/03,boxart/original/front/45987-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...


In [5]:
# Count records by column
allsys_df.count()

gdb_id          29759
Title           29759
Release_Date    27170
Thumb           29585
Platform        29759
URL             29759
dtype: int64

In [6]:
# Remove rows with empty thumb backlink and recount
allsys_df['Thumb'].replace('', np.nan, inplace=True)

allsys_df.dropna(subset=['Thumb'], inplace=True)

allsys_df.count()

gdb_id          29585
Title           29585
Release_Date    27146
Thumb           29585
Platform        29585
URL             29585
dtype: int64

In [12]:
# Rename column from Title to Name
allsys_df = allsys_df.rename(columns = {'Title':'Name'})
allsys_df.head()

,gdb_id,Name,Release_Date,Thumb,Platform,URL
0,3373,Dark Arena,1/18/02,boxart/original/front/3373-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...
1,3374,Dave Mirra Freestyle BMX 2,11/25/01,boxart/original/front/3374-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...
2,3375,Davis Cup Tennis,10/14/02,boxart/original/front/3375-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...
3,198,Classic NES Series: Zelda II: The Adventure of...,10/25/04,boxart/original/front/198-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...
4,45987,Dragon Drive: World D Break,7/18/03,boxart/original/front/45987-1.jpg,GBA,http://thegamesdb.net/banners/boxart/original/...


In [ ]:
# # Filter to select systems if desired
# systems = [GBA, ...]
# gapminder.Platform.isin(systems)

In [9]:
# Read in master dataset with sales and critic scores
vgsales_df = pd.read_csv('Video_Game_Sales.csv')
vgsales_df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.54,76.0,51.0,8.0,324.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,3.29,35.57,82.0,73.0,8.3,712.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.95,3.28,2.95,32.78,80.0,73.0,8.0,193.0,E
4,Pokemon Red/Pokemon Blue,G,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN


In [14]:
# Merge URL dataframe to vgsales dataframe on Name and Platform
result = pd.merge(vgsales_df,
                 allsys_df[['Name', 'Platform', 'URL']],
                 on=['Name', 'Platform'])
result.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating,URL
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.54,76.0,51.0,8.0,324.0,E,http://thegamesdb.net/banners/boxart/original/...
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,http://thegamesdb.net/banners/boxart/original/...
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,3.29,35.57,82.0,73.0,8.3,712.0,E,http://thegamesdb.net/banners/boxart/original/...
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.95,3.28,2.95,32.78,80.0,73.0,8.0,193.0,E,http://thegamesdb.net/banners/boxart/original/...
4,Tetris,G,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26,NaN,NaN,NaN,NaN,NaN,http://thegamesdb.net/banners/boxart/original/...


In [19]:
# Check counts
result.count()

Name               9519
Platform           9519
Year_of_Release    9515
Genre              9519
Publisher          9519
NA_Sales           9519
EU_Sales           9519
JP_Sales           9519
Other_Sales        9519
Global_Sales       9519
Critic_Score       6142
Critic_Count       6142
User_Score         5649
User_Count         5649
Rating             7251
URL                9519
dtype: int64

In [20]:
# Sort and filter by top 2600 NA_Sales
result = result.sort_values('NA_Sales', ascending=False)
result.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating,URL
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.54,76.0,51.0,8.0,324.0,E,http://thegamesdb.net/banners/boxart/original/...
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,http://thegamesdb.net/banners/boxart/original/...
8,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31,NaN,NaN,NaN,NaN,NaN,http://thegamesdb.net/banners/boxart/original/...
4,Tetris,G,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26,NaN,NaN,NaN,NaN,NaN,http://thegamesdb.net/banners/boxart/original/...
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,3.29,35.57,82.0,73.0,8.3,712.0,E,http://thegamesdb.net/banners/boxart/original/...


In [23]:
# Cut to top 2600 sales
top2600 = result[:2600]
top2600.count()

Name               2600
Platform           2600
Year_of_Release    2600
Genre              2600
Publisher          2600
NA_Sales           2600
EU_Sales           2600
JP_Sales           2600
Other_Sales        2600
Global_Sales       2600
Critic_Score       1853
Critic_Count       1853
User_Score         1875
User_Count         1875
Rating             2041
URL                2600
dtype: int64

In [24]:
# create list from URLs
top_piclist = top2600["URL"].tolist()
top_piclist

['http://thegamesdb.net/banners/boxart/original/front/3293-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/140-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/346-2.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/10616-2.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/169-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/3194-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/8871-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/170-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/3495-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/136-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/6859-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/1726-1.png',
 'http://thegamesdb.net/banners/boxart/original/front/11740-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/6837-1.jpg',
 'http://thegamesdb.net/banners/boxart/original/front/13353-1.jpg

In [26]:
# check link quality in list

print(top_piclist[3])

http://thegamesdb.net/banners/boxart/original/front/10616-2.jpg


In [35]:
# Break list of links into pages of 50
paginated_urls = {}
url_list = []
count = 0
for url in top_piclist:
    if count % 50 == 0:
        paginated_urls[count // 50] = url_list
        url_list = []
    url_list.append(url)
    count += 1
paginated_urls[(count // 50) + 1] = url_list
url_list = []
paginated_urls

{0: [],
 1: ['http://thegamesdb.net/banners/boxart/original/front/3293-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/140-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/346-2.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/10616-2.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/169-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/3194-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/8871-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/170-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/3495-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/136-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/6859-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/1726-1.png',
  'http://thegamesdb.net/banners/boxart/original/front/11740-1.jpg',
  'http://thegamesdb.net/banners/boxart/original/front/6837-1.jpg',
  'http://thegamesdb.net/banners/boxart/

In [47]:
# Test open save a single image before looping through all

# fd = urlopen(Request('http://thegamesdb.net/banners/boxart/original/front/1292-2.jpg', headers={'User-Agent': 'Chrome'}))
# im = Image.open(fd)
# im.thumbnail([120,120])
# im = im.convert("RGB")
# im.save(f'tiles/{url[52:]}',format='JPEG')

In [36]:
# Loop through pages to open, resize, and save cover art images to tiles folder

for i in range(1, 54):
    for url in paginated_urls[i]:
        try:
            fd = urlopen(Request(url, headers={'User-Agent': 'Chrome'}))
            im = Image.open(fd)
            im.thumbnail([120,120])
            im = im.convert("RGB")
            im.save(f'tiles/{url[52:]}',format='JPEG')
        except KeyError:
            print('err')
        except ConnectionResetError:
            print('err1')
    print(i)
print('DONE')

53
DONE
